In [2]:
import os
import numpy as np
import pandas as pd
from tqdm import tqdm
from joblib import load, dump, Parallel, delayed

from scipy.stats import pearsonr

In [9]:
preds = []
targets = []

for fold in range(4):
    preds.append(load("../ckpts/ffnn_seed_2022/fold-{}-ohem-1-CyclicLR-lr-0.008-hidden-64-dropout-0.15-l2-0.02-l1-0.0002-epoch-40-direction-1/val_preds_fold_{}.pkl".format(fold, fold)))
    targets.append(load("../ckpts/ffnn_seed_2022/fold-{}-ohem-1-CyclicLR-lr-0.008-hidden-64-dropout-0.15-l2-0.02-l1-0.0002-epoch-40-direction-1/val_targets_fold_{}.pkl".format(fold, fold)))

In [10]:
preds = np.concatenate(preds).flatten()
targets = np.concatenate(targets).flatten()

corr, _ = pearsonr(preds, targets)

print(corr)

0.15904659602638294
